In [1]:
import os
from dotenv import load_dotenv

from pathlib import Path
import sys

PATH_PROJECT_ROOT_DIR = Path(os.getcwd()).resolve().parent
PATH_SRC_DIR = os.path.join(PATH_PROJECT_ROOT_DIR, "src/")

sys.path.append(str(PATH_SRC_DIR))

In [2]:
from pipeline_scripts.api_twitter_helper import fetch_recent_tweets
from pipeline_scripts.nlp_model_helper import sentiment_analysis


In [3]:
# Safe mode, you can write your token in a .env or specify it directly into the function.
load_dotenv()
## X API Bearer Token
BEARER_TOKEN = os.getenv("BEARER_TOKEN") 

## Hugging Face Token
TOKEN_HF = os.getenv("TOKEN_HF") 

## Broker (Alpaca) API credentials
BROKER_TOKEN = os.getenv("BROKER_API_KEY") 
BROKER_SECRET = os.getenv("BROKER_API_SECRET")

ACCOUNTS = None

MODEL_NAME = "StephanAkkerman/FinTwitBERT-sentiment" 
TICKER = "ETH/USD"
ASSET_NAME =  "Ethereum" # If you do not want to search tweets for company ticker name you can specify it here

In [4]:
from pipeline_scripts.api_twitter_helper import fetch_recent_tweets

input_data = fetch_recent_tweets(bearer_token=BEARER_TOKEN,
                                 accounts=ACCOUNTS if ACCOUNTS else None,
                                 asset=ASSET_NAME if ASSET_NAME else TICKER,
                                 max_results=10)

Query: Ethereum -is:retweet -is:reply
Error fetching tweets: 500 Internal Server Error


In [ ]:

res = sentiment_analysis(token_hf=TOKEN_HF, model_to_use=MODEL_NAME, data=input_data)


Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
from pipeline_scripts.trading_signal import generate_trading_signal
ACTION = generate_trading_signal(df=res, buy_threshold=0.3, sell_threshold=-0.3)
ACTION


Action: HOLD — Sentiment does not meet buy or sell criteria.


(0.0073517471551895145, 'HOLD')

In [ ]:
from pipeline_scripts.trader_helper import AlpacaTrader
trader = AlpacaTrader(
    api_key=BROKER_TOKEN,
    secret_key=BROKER_SECRET)
print("Account ID: ", trader.trading_client.get_account().account_number, "\nPortfolio value: ",trader.trading_client.get_account().portfolio_value,"$") 

# trader.trading_client.cancel_orders()



In [ ]:
trader.trading_client.get_clock()['is_open']  # Check if the market is open


In [ ]:
trader.execute_action(
    symbol=TICKER,
    asset_type="stock",
    action=ACTION[1],
    qty= 1,
    threshold_pct=5.0,
    timeframe_unit="Hour",
    limit=10,
)

trader.trading_client.get_orders()

[]